# Run the CICI SAFE vSDX

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-819993'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME='yjyao_jupyter1'
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
export CLIENT_RESERVATION_ID='c3f9c003-888b-4dbd-86c6-e79a388d136a'

#Set the working directory
export WORKING_DIR=${HOME}/work/ExoGENI-Client

#Set GENI CERT file
export GENI_CERT=${HOME}/work/geni.pem

export SDX_PUBLIC_IP='129.114.109.69'

In [2]:
#need to install exogeni phython script in the container
#python delete ${DATAPLANE_SLICE_NAME}

#### Start ExoGENI Client

In [3]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME=${USERNAME}-client2
CLIENT_STACK_NAME=${USERNAME}-client2
GIT_BRANCH='tridentcom'
SLICE_NAME=c1-${USERNAME}
IP_PREFIX='192.168.20.1/24'
CLIENT_SITE='UNF'
SAFE_KEYPAIR='key_p11'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template ${WORKING_DIR}/client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --parameter "slice_name=${SLICE_NAME}"\
        --parameter "ip_prefix=${IP_PREFIX}"\
        --parameter "client_site=${CLIENT_SITE}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME}

Creating SDX client. This will take several minutes!
ERROR: The Stack (yaoyj11-client2) already exists.


: 1

In [ ]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME}`

CLIENT_PUBLIC_IP=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP


#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

In [5]:
#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem

echo Copying GENI cert to cc@$CLIENT_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

Copying GENI cert to cc@129.114.108.105:~/geni.pem
Done


In [ ]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo tail -F /home/cc/boot.log

[INFO] Installing artifact /root/.m2/repository/commons-cli/commons-cli/1.4/commons-cli-1.4.jar to /home/cc/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/repo/commons-cli/commons-cli/1.4/commons-cli-1.4.jar
[INFO] Installing artifact /home/cc/CICI-SAFE/SDX-Simple/SAFE_SDX/target/exoplex.sdx-simple-0.1-SNAPSHOT.jar to /home/cc/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/repo/safe/exoplex.sdx-simple/0.1-SNAPSHOT/exoplex.sdx-simple-0.1-SNAPSHOT.jar
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time: 1:22.482s
[INFO] Finished at: Fri Mar 01 21:00:10 UTC 2019
[INFO] Final Memory: 33M/1100M
[INFO] ------------------------------------------------------------------------
Boot Script Done!


#### Create SAFE keypair

In [ ]:
SAFE_KEYPAIR="client2"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    'sudo docker exec -itd safe /bin/bash /root/safe_keygen.sh '${SAFE_KEYPAIR}' /root/safe/safe-server/src/main/resources/prdnsmall'

res=$(curl http://${CLIENT_PUBLIC_IP}:7777/whoami \
     -H "Cntent-Type:application/json" \
     -d '{"principal":"'${SAFE_KEYPAIR}'","methodParams":[]}'
     )
     
principalId=$(echo $res | cut -d"'" -f 2)

echo principalId for ${SAFE_KEYPAIR} is $principalId


In [ ]:
LOCAL_CLIENT_CONFIG=${WORKING_DIR}/client1.conf

REMOTE_CLIENT_CONFIG=/home/cc/client1.conf
REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

cat << EOF > $LOCAL_CLIENT_CONFIG
config {
slicename="${SLICE_NAME}"
type="client"
sshkey="/root/.ssh/id_rsa"
safekey="${SAFE_KEYPAIR}"
safeserver="127.0.0.1"
safe=true
serverinslice=false
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
resourcedir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/"
ipprefix="${IP_PREFIX}"
serverurl="http://${SDX_PUBLIC_IP}:8888/"
routersite="${CLIENT_SITE}"
}
EOF

echo Copying SDX config to cc@$CLIENT_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_CLIENT_CONFIG \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_CLIENT_CONFIG}
echo Done

In [ ]:
echo Start Client Slice

echo $REMOTE_CLIENT_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxClientSliceServer -c $REMOTE_CLIENT_CONFIG

echo "Slice creation done"
#cd ${HOME}/work/CICI-SAFE/SDX-Simple
#./scripts/build.sh
#./scripts/createclientslice.sh -c $LOCAL_CLIENT_CONFIG

#### Client 2 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'stitch CNode1 192.168.20.2 192.168.20.1/24'"

#### Client 2 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'route 192.168.20.1/24 192.168.20.2'"

#### Connect client 1 and client 2

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'link 192.168.20.1/24 192.168.10.1/24'"

#### Undo stitching

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'unstitch CNode1'"

# Please Cleanup Your Resources!!!

#### Delete Dataplane Slice

#### Delete Client Stack

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME}
openstack stack delete -y --wait ${CLIENT_STACK_NAME}
echo Stack ${CLIENT_STACK_NAME} deleted